In [33]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split

# Load and validate data

# Split

In [44]:
df_images = pd.read_csv('images_tiktok.csv', header=None, dtype={0: str}).rename(columns={0: "id", 1: 'image_path'})
text_tiktok =  pd.read_csv('text_tiktok.csv', header=None, dtype={1: str}).rename(columns={0: "label", 1: 'id', 2: 'text'})
text_tiktok_none_hashtag = pd.read_csv('text_tiktok.csv', header=None, dtype={1: str}, usecols=[1, 2]).rename(columns={1: 'id', 2: 'text_non_hastag'})

In [45]:
raw_dataset = pd.merge(left=text_tiktok, 
                        right=text_tiktok_none_hashtag, 
                        how='inner', 
                        on='id')
train_set_1, remain = train_test_split(raw_dataset, test_size=0.5, random_state=2022, stratify=raw_dataset['label'])
train_set_2, test_set = train_test_split(remain, test_size=0.3, random_state=2022, stratify=remain['label'])

In [46]:
train_set_1.to_csv('train_set_1.csv', index=False)
train_set_2.to_csv('train_set_2.csv', index=False)
test_set.to_csv('test_set.csv', index=False)

# Join with image

In [52]:
train_set_1_image = pd.merge(left=df_images, 
                            right=train_set_1[['label', 'id']], 
                            how='inner', 
                            on='id')
train_set_2_image = pd.merge(left=df_images, 
                            right=train_set_2[['label', 'id']], 
                            how='inner', 
                            on='id')
test_set_image = pd.merge(left=df_images, 
                            right=test_set[['label', 'id']], 
                            how='inner', 
                            on='id')

In [53]:
train_set_1_image.to_csv('train_set_1_image.csv', index=False)
train_set_2_image.to_csv('train_set_2_image.csv', index=False)
test_set_image.to_csv('test_set_image.csv', index=False)

In [54]:
train_set_1_image.shape

(39913, 3)

In [55]:
train_set_2_image.shape

(27115, 3)

In [56]:
test_set_image.shape

(12643, 3)

In [57]:
39913+27115+12643

79671

In [58]:
df_images.shape

(79671, 2)

In [59]:
train_set_1_image

,id,image_path,label
0,7068622099056823579,images/7068622099056823579_500.jpeg,1
1,7068622099056823579,images/7068622099056823579_3576.jpeg,1
2,7068622099056823579,images/7068622099056823579_1750.jpeg,1
3,7068622099056823579,images/7068622099056823579_2000.jpeg,1
4,7068622099056823579,images/7068622099056823579_00.jpeg,1
...,...,...,...
39908,6920036645060267265,images/6920036645060267265_250.jpeg,0
39909,6920036645060267265,images/6920036645060267265_1250.jpeg,0
39910,6920036645060267265,images/6920036645060267265_750.jpeg,0
39911,6995479941269802267,images/6995479941269802267_00.jpeg,0


In [1]:
index_lst = [(0, 5000), (5000, 10000), (10000, 15000), (15000, 20000), (20000, 25000), (25000, 30000), (30000, 35000), (35000, 39913)]

In [7]:
for i, tup_idx in enumerate(index_lst):
    if i != 0:
        checkpoint_path = f'model_checkpoint/checkpoint_{index_lst[i-1][0]}_{index_lst[i-1][1]}.h5'
        print(checkpoint_path)
        print(f'model_checkpoint/checkpoint_{tup_idx[0]}_{tup_idx[1]}.h5')
        print()

model_checkpoint/checkpoint_0_5000.h5
model_checkpoint/checkpoint_5000_10000.h5

model_checkpoint/checkpoint_5000_10000.h5
model_checkpoint/checkpoint_10000_15000.h5

model_checkpoint/checkpoint_10000_15000.h5
model_checkpoint/checkpoint_15000_20000.h5

model_checkpoint/checkpoint_15000_20000.h5
model_checkpoint/checkpoint_20000_25000.h5

model_checkpoint/checkpoint_20000_25000.h5
model_checkpoint/checkpoint_25000_30000.h5

model_checkpoint/checkpoint_25000_30000.h5
model_checkpoint/checkpoint_30000_35000.h5

model_checkpoint/checkpoint_30000_35000.h5
model_checkpoint/checkpoint_35000_39913.h5



In [ ]:
import gc

for i, tup_idx in enumerate(index_lst):
    if i != 0:
        start_idx = tup_idx[0]
        end_idx = tup_idx[1]
        num_ep = 10
        steps_per_epoch = 256
        validation_steps = 32

        trainer = image_binary_classify_keras()
        checkpoint_path = f'model_checkpoint/checkpoint_{index_lst[i-1][0]}_{index_lst[i-1][1]}.h5'
        trainer.init_model(checkpoint_path=checkpoint_path)
#         trainer.init_model()

        train_label_df = train_label_raw.iloc[start_idx:end_idx]
        trainer.generate_train_valid(img_path_arr=train_label_df['image_path'], label_arr=train_label_df['label'])
        trainer.train_model(num_epochs=num_ep, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)

        # save model
        os.makedirs('model_checkpoint', exist_ok=True)
        trainer.save_model(f'model_checkpoint/checkpoint_{start_idx}_{end_idx}.h5')
        gc.collect()